# Kaggle Competition
## Google Brain - Ventilator Pressure Prediction
### Simulate a ventilator connected to a sedated patient's lung

<https://www.kaggle.com/c/ventilator-pressure-prediction>

I can take inspiration from <https://www.kaggle.com/yasufuminakama/ventilator-pressure-lstm-starter>

## Notes

- maybe from test `breath_id` I can infer the type of profile
- I can predict in the frequency domain, with a fourier transform

In [3]:
config = {
    'nn_depth': 7,
    'layers_width': 80,
    'dropout_p': 0.2,
    'learning_rate': 10e-3,
    'batch_size': 64,
}
f'{config}'

"{'nn_depth': 7, 'layers_width': 80, 'dropout_p': 0.2, 'learning_rate': 0.01, 'batch_size': 64}"

In [1]:
# Basic
import numpy as np
import pandas as pd
import pickle
import gzip
from pathlib import Path

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Clustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

# Various regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# PyTorch
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

# Hyperparametes optimization
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

# Visualization
import ipywidgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

base_path = Path('')
input_path = base_path / 'input'
data_path = base_path / 'data'
plot_path = base_path / 'plots'
model_path = base_path / 'models'
Path.mkdir(data_path, exist_ok=True)
Path.mkdir(plot_path, exist_ok=True)
Path.mkdir(model_path, exist_ok=True)


In [6]:
should_fit = True

time_delta_scaler, pressure_scaler = MinMaxScaler(), MinMaxScaler()

df = pd.read_csv(input_path / 'train.csv')
df = df.copy()
# Remove the expiration phase
df = df[df['u_out'] == 0].drop(columns='u_out')

# Adding time_delta
df['time_delta'] = df['time_step'] - df.groupby('breath_id')['time_step'].shift(1)
df['time_delta'] = df['time_delta'].clip(upper=0.04) # some time_delta are really huge, probably capping them is a good idea
df['time_delta'] = df['time_delta'].fillna(df['time_delta'].mean()) # set first time-delta as mean

if should_fit:
    # Prepare for rescaling. should be done only on training, but meh.
    time_delta_scaler.fit(df['time_delta'].values.reshape(-1, 1))
    pressure_scaler.fit(df['pressure'].values.reshape(-1, 1))

# Scaling
df['time_delta'] = time_delta_scaler.transform(df[['time_delta']].values)
if 'pressure' in df:
    df['pressure'] = pressure_scaler.transform(df[['pressure']].values)
else:
    df['pressure'] = 0
df['u_in'] = df['u_in'] / 100
df['R'] = df['R'].map({5:0, 20:0.5, 50:1})
df['C'] = df['C'].map({10:0, 20:0.5, 50:1})

In [7]:
df

,id,breath_id,R,C,time_step,u_in,pressure,time_delta
0,1,1,0.5,1.0,0.000000,0.000833,0.115911,0.187894
1,2,1,0.5,1.0,0.033652,0.183830,0.116965,0.258445
2,3,1,0.5,1.0,0.067514,0.225093,0.146470,0.282956
3,4,1,0.5,1.0,0.101542,0.228088,0.204426,0.302313
4,5,1,0.5,1.0,0.135756,0.253559,0.211802,0.323983
...,...,...,...,...,...,...,...,...
6035945,6035946,125749,1.0,0.0,0.834147,0.018694,0.469968,0.231456
6035946,6035947,125749,1.0,0.0,0.867574,0.021544,0.464700,0.232097
6035947,6035948,125749,1.0,0.0,0.900917,0.013044,0.476291,0.222348
6035948,6035949,125749,1.0,0.0,0.934309,0.017338,0.468915,0.227974


# Preprocessing

In [3]:
def flatten_df(df, time_delta_scaler, pressure_scaler, should_fit):
    # Remove the expiration phase
    df = df[((df['id'] - 1) % 80) < 32].copy()

    # adding time_delta
    df['time_delta'] = df['time_step'] - df.groupby('breath_id')['time_step'].shift(1)
    df['time_delta'] = df['time_delta'].clip(upper=0.04) # some time_delta are really huge, probably capping them is a good idea
    df['time_delta'] = df['time_delta'].fillna(df['time_delta'].mean()) # set first time-delta as mean

    if should_fit:
        # Prepare for rescaling. should be done only on training, but meh.
        time_delta_scaler.fit(df['time_delta'].values.reshape(-1, 1))
        pressure_scaler.fit(df['pressure'].values.reshape(-1, 1))

    # Scaling
    df['time_delta'] = time_delta_scaler.transform(df[['time_delta']].values)
    if 'pressure' in df:
        df['pressure'] = pressure_scaler.transform(df[['pressure']].values)
    else:
        df['pressure'] = 0
    df['u_in'] = df['u_in'] / 100
    df['R'] = df['R'].map({5:0, 20:0.5, 50:1})
    df['C'] = df['C'].map({10:0, 20:0.5, 50:1})

    # Transpose
    pressure_df = df.groupby('breath_id')['pressure'].apply(lambda dff: dff.reset_index(drop=True)).unstack()

    R_C = df.groupby('breath_id')[['R', 'C']].mean()
    time_out = df[df['u_out'] == 0].groupby('breath_id')[['time_step', 'id']].max()
    time_out['id'] = (time_out['id'] - 1) % 80 + 1

    df = df.groupby('breath_id')[['u_in', 'time_delta']].apply(lambda dff: dff.reset_index(drop=True)).unstack()
    df['R'] = R_C['R']
    df['C'] = R_C['C']
    df['index_out'] = time_out['id']
    return df.reset_index(), pressure_df.reset_index()


time_delta_scaler, pressure_scaler = MinMaxScaler(), MinMaxScaler()

df = pd.read_csv(input_path / 'train.csv')
df, pressure_df = flatten_df(df, time_delta_scaler, pressure_scaler, should_fit=True)

df_test = pd.read_csv(input_path / 'test.csv')
df_test, pressure_test = flatten_df(df_test, time_delta_scaler, pressure_scaler, should_fit=False)

## Clustering

In [4]:
n_clusters = 20
clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(df['u_in']) # doesn't work well
df['cluster'] = clustering.predict(df['u_in'])
df_test['cluster'] = clustering.predict(df_test['u_in'])
#dff = df.sample(10000)
#clustering = DBSCAN().fit(dff['u_in'])
#dff['cluster'] = clustering.labels_
#dff['cluster'] +=1

## Train Validation split

In [5]:
df_train, df_valid, pressure_train, pressure_valid = train_test_split(df, pressure_df, stratify=df[['cluster']], test_size=0.2, random_state=0)

for dff in [df_train, pressure_train, df_valid, pressure_valid]:
    dff.index = dff['breath_id'].values
    dff.sort_index(inplace=True)

with open(data_path / 'flat_train.pickle', 'wb') as handle:
    pickle.dump((df_train, pressure_train), handle)
with open(data_path / 'flat_valid.pickle', 'wb') as handle:
    pickle.dump((df_valid, pressure_valid), handle)
with open(data_path / 'flat_test.pickle', 'wb') as handle:
    pickle.dump((df_test, pressure_test), handle)
with open(data_path / 'reverse_transform.pickle', 'wb') as handle:
    pickle.dump((time_delta_scaler, pressure_scaler), handle)

# Data Exploration

In [6]:
with open(data_path / 'flat_train.pickle', 'rb') as handle:
    df, pressures = pickle.load(handle)

## Time Deltas

In [7]:
# fig = go.Figure()
# fig.add_trace(go.Histogram(x=time_deltas, nbinsx=500))
# fig.write_html(plot_path / 'timedeltas.html') # That's a pretty weird distribution

## Clustering

In [8]:
# R and C stratification
display(df[['R', 'C']].value_counts(normalize=True).unstack())
display(df_test[['R', 'C']].value_counts(normalize=True).unstack())

In [9]:
# Plot clustering PCA
reduced = PCA(n_components=2).fit_transform(df['u_in'])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=reduced[:,0], y=reduced[:,1], marker_color=df['cluster'], mode='markers'
))
fig.write_html(plot_path / 'clustering_DBSCAN_pca.html')

# Plot clustering breath profiles
fig = make_subplots(rows=5, cols=4,
    subplot_titles=[f'Cluster {cluster} N={size}' for cluster, size in df['cluster'].value_counts().sort_index().iteritems()]
)
for cluster in tqdm(df['cluster'].sort_values().unique()):
    dff = df[df['cluster'] == cluster]
    dff = dff.sample(n=min(400, len(dff))) # draw 400 traces for each cluster
    for _, row in dff.iterrows():
        u_in = row['u_in']
        times = pd.DataFrame(row['time_delta'])
        times = np.cumsum(time_delta_scaler.inverse_transform(times))
        fig.add_trace(
            go.Scatter(x=times, y=u_in, opacity=0.05, marker={'color': '#0000FF'}),
            row=cluster//4+1, col=cluster%4+1
        )
fig.update_layout(
    showlegend=False,
)
fig.write_html(plot_path / 'clustering_DBSCAN_traces.html')

In [10]:
pd.DataFrame({
    'train': df[['cluster', 'breath_id']].groupby('cluster').count().breath_id / len(df),
    'test': df_test[['cluster', 'breath_id']].groupby('cluster').count().breath_id / len(df_test),
}) *100

In [11]:
dff = df.sample(2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=dff['breath_id'], y=dff['R'], mode='markers'))
fig.add_trace(go.Scatter(x=dff['breath_id'], y=dff['C'] + 2, mode='markers'))
fig.add_trace(go.Scatter(x=dff['breath_id'], y=dff['cluster'] + 4, mode='markers'))
fig.write_html('plots/breath_id_R_C_clusters.html') # R, C and clusters are randomly distributed, no id hacking possible

# Pressure Prediction

## Pytorch Preparation

In [8]:
class SequencesDataset(Dataset):
    def __init__(self, df_path):
        with open(df_path, 'rb') as handle:
            df, pressures = pickle.load(handle)
        self.X = df[['u_in', 'time_delta', 'R', 'C']].values
        self.Y = pressures.drop(columns='breath_id').values
        for index in range(7):
            df[index] = (df['index_out'] > (25 + index)).astype(int)
        self.extra = df[['breath_id', 'index_out', 'cluster'] + list(range(7))].values

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.Tensor(self.X[idx]).to(device), torch.Tensor(self.Y[idx]).to(device), torch.Tensor(self.extra[idx]).to(device)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

train_data = SequencesDataset(data_path / 'flat_train.pickle')
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

valid_data = SequencesDataset(data_path / 'flat_valid.pickle')
valid_dataloader = DataLoader(valid_data, batch_size=64, shuffle=True)

Using cpu device


## Model Definition

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self, layers_width, dropout_p):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(66, layers_width), # 32 u_in, 32 time_delta, R, C
            nn.ReLU(),
            nn.Dropout(p=dropout_p),

            nn.Linear(layers_width, layers_width),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),

            nn.Linear(layers_width, layers_width),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),

            nn.Linear(layers_width, layers_width),
            nn.ReLU(),

            nn.Linear(layers_width, 32),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

layers_width = 70
dropout_p = 0
model = NeuralNetwork(layers_width, dropout_p).to(device)
print(model)
writer = SummaryWriter('runs/dropout_0.8__width_90__height_7')
writer.add_graph(model, next(iter(train_dataloader))[0])

## Training

In [3]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [22]:
learning_rate = 3e-4
batch_size = 64
epochs = 3000

loss_function = nn.MSELoss(reduction='sum')

def fixed_loss_function(Y, prediction, extra):
    prediction[:,-7:] *= extra[:,-7:] # put to 0 prediction during expiration phase in order to 0 their grad
    Y[:,-7:] *= extra[:,-7:] # put to 0 target during expiration phase in order to not increase the loss
    return loss_function(prediction, Y) / (extra[:,-7:].sum() + 25 * Y.shape[0]) # divide only for the number of cases in inspiration phase

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
size = len(train_dataloader.dataset)
min_valid_loss = 1

for epoch in tqdm(range(epochs)):
    model.train()
    train_loss = 0
    for X, Y, extra in train_dataloader:
        prediction = model(X)
        loss = fixed_loss_function(prediction, Y, extra)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for X, Y, extra in valid_dataloader:
            prediction = model(X)
            valid_loss += fixed_loss_function(prediction, Y, extra).item()
    if min_valid_loss > valid_loss / len(valid_dataloader):
        min_valid_loss = valid_loss / len(valid_dataloader)
        best_model_parameters = model.state_dict()

    writer.add_scalars('loss/train_valid', {
        'train': train_loss / len(train_dataloader),
        'valid': valid_loss / len(valid_dataloader),
    }, epoch)

model.load_state_dict(best_model_parameters)
torch.save(model, model_path / 'model.pth')

# Prediction Analysis

In [30]:
with open(data_path / 'reverse_transform.pickle', 'rb') as handle:
    time_delta_scaler, pressure_delta_scaler = pickle.load(handle)
valid_final_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)
metric_function = nn.L1Loss(reduction='none')

model.eval()
with torch.no_grad():
    X, Y, extra = next(iter(valid_final_dataloader))
    prediction = model(X)
    full_metric = metric_function(prediction, Y).cpu().numpy() * pressure_delta_scaler.data_range_.item()
X = X.cpu().numpy()
extra = extra.cpu().numpy()
full_metric[:,-7:][extra[:,-7:] == 0] = np.nan
full_metric = pd.DataFrame(full_metric, index=extra[:,0])
valid_error = full_metric.sum().sum() / full_metric.notna().sum().sum()
print(f'Error on validation: {valid_error}')

R_C_cluster = pd.DataFrame({'R': X[:,-2], 'C': X[:,-1], 'cluster': extra[:,2]}, index=extra[:,0])

## Error by R C

In [31]:
fig = make_subplots(rows=3, cols=3, subplot_titles=[f'R={R} C={C}' for R in [0, 0.5, 1] for C in [0, 0.5, 1]])
for R in [0, 0.5, 1]:
    for C in [0, 0.5, 1]:
        filtered_metric = full_metric[(R_C_cluster['R'] == R) & (R_C_cluster['C'] == C)]
        fig.add_trace(
            go.Scatter(y=filtered_metric.mean()),
            row=int(R*2+1), col=int(C*2+1)
        )
fig.update_layout(height=600, margin={'t':30, 'l':30, 'b':30, 'r':30}, showlegend=False)
fig

## Error by Cluster

In [ ]:
fig = make_subplots(rows=5, cols=4,
    subplot_titles=[f'Cluster {int(cluster)} N={size}' for cluster, size in R_C_cluster['cluster'].value_counts().sort_index().iteritems()]
)
for cluster in R_C_cluster['cluster'].sort_values().unique():
    cluster = int(cluster)
    filtered_metric = full_metric[R_C_cluster['cluster'] == cluster]
    fig.add_trace(
        go.Scatter(y=filtered_metric.mean()),
        row=cluster//4+1, col=cluster%4+1
    )
fig.update_layout(height=600, margin={'t':30, 'l':30, 'b':30, 'r':30}, showlegend=False)
fig

## Plot single Breath

In [ ]:
with open('output/flat_valid.pickle', 'rb') as handle:
    dff, pressures = pickle.load(handle)

def plot_breath(breath_id, df, pressures, prediction=None):
    df.index = df['breath_id']
    pressures.index = pressures['breath_id']
    row = df.loc[breath_id]
    u_in = row['u_in']
    times = pd.DataFrame(row['time_delta'])
    times = np.cumsum(time_delta_scaler.inverse_transform(times))
    pressures = pressures.loc[breath_id][1:]
    prediction = prediction[0].numpy()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=times, y=u_in, opacity=0.5, marker={'color': '#0000FF'}, name='u_in', mode='lines+markers'
    ))
    fig.add_vline(x=times[int(row['index_out'])-1].item())
    fig.add_trace(go.Scatter(
        x=times, y=prediction, opacity=0.5, marker={'color': '#FF3333'}, name='prediction', mode='lines+markers'
    ))
    fig.add_trace(go.Scatter(
        x=times, y=pressures, opacity=0.7, marker={'color': 'red'}, name='target', mode='lines+markers'
    ))
    fig.update_layout(
        hovermode='x unified',
        title=f"breath_id: {row['breath_id'].item()} R={row['R'].item()} C={row['C'].item()}"
    )
    return fig


plot_breath(dff.index[6], dff, pressures, prediction)

In [ ]:
writer.close()

## Old Visualization

In [ ]:
def plot_breath(df, breath_index, prediction=None):
    fig = go.Figure()
    df['prediction'] = prediction
    one = df[df['breath_id'] == breath_index]
    fig.add_trace(go.Scatter(
        x=one['time_step'], y=one['u_in'], opacity=0.5, marker={'color': '#0000FF'}, name='u_in', mode='lines+markers'
    ))
    fig.add_trace(go.Scatter(
        x=one['time_step'], y=one['u_out']*20, opacity=0.5, marker={'color': 'green'}, name='u_out', mode='lines+markers'
    ))
    fig.add_trace(go.Scatter(
        x=one['time_step'], y=one['prediction'], opacity=0.5, marker={'color': '#FF3333'}, name='prediction', mode='lines+markers'
    ))
    fig.add_trace(go.Scatter(
        x=one['time_step'], y=one['pressure'], opacity=0.7, marker={'color': 'red'}, name='target', mode='lines+markers'
    ))
    fig.update_layout(
        hovermode='x unified',
        title=f"Breath ID: {breath_index} R={one['R'].iloc[0]} C={one['C'].iloc[0]}"
    )
    return fig

metrics = _
breath_index = df_valid['breath_id'].unique()[0]

# breath_index = metrics.index[-500]
# fig = plot_breath(df_valid, breath_index)
fig = plot_breath(df_train, 4)
fig.show()
# weird breaths:
# - long time-delta: 24127, 55851, 72104
# - negative pressure: 542, 77803, 112036, 45099

# Test

In [ ]:
# Load data and model
with open('output/reverse_transform.pickle', 'rb') as handle:
    _, pressure_scaler = pickle.load(handle)
test_data = SequencesDataset('output/flat_test.pickle')
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)
model = torch.load('output/model.pth')

# Compute predictions
with torch.no_grad():
    X, Y, extra = next(iter(test_dataloader))
    prediction = model(X)

# Inverse preprocessing to reconstruct pressure column
pressures = pd.DataFrame(prediction.numpy())
for index in range(32, 80):
    pressures[index] = 0
pressures = pressures.transpose()
pressures = pd.concat([pressures[column] for column in pressures])
pressures = pressure_scaler.inverse_transform(pd.DataFrame(pressures))
submission = pd.DataFrame({'id': range(1, len(pressures) + 1), 'pressure': pressures[:,0]})

# Save submission as .csv and .gz
submission.to_csv('output/submission.csv', index=False)
with open('output/submission.csv', 'rb') as handle:
    text = handle.read()
with gzip.open('output/submission.gz', 'wb') as handle:
    handle.write(text)